In [1]:
import bonesis
import bonesis.aeon
import biodivine_aeon
import os
import csv

In [2]:
aeon = "ABA_A.aeon"
data_loc = "ABA_A.tsv"

In [3]:
aeon_bn = biodivine_aeon.BooleanNetwork.from_file(aeon)

aeon_bn.explicit_parameter_count()

rg = aeon_bn.to_graph()


print(len(rg.variable_names()))

data = {}
for node in rg.variable_names():
    data[node + "_OFF"] = {node:0}
    data[node + "_ON"] = {node:1}


print(data)

86
{'ABA_OFF': {'ABA': 0}, 'ABA_ON': {'ABA': 1}, 'ABH1_OFF': {'ABH1': 0}, 'ABH1_ON': {'ABH1': 1}, 'ABI1_OFF': {'ABI1': 0}, 'ABI1_ON': {'ABI1': 1}, 'ABI2_OFF': {'ABI2': 0}, 'ABI2_ON': {'ABI2': 1}, 'ADPRc_OFF': {'ADPRc': 0}, 'ADPRc_ON': {'ADPRc': 1}, 'ARP_Complex_OFF': {'ARP_Complex': 0}, 'ARP_Complex_ON': {'ARP_Complex': 1}, 'Actin_Reorganization_OFF': {'Actin_Reorganization': 0}, 'Actin_Reorganization_ON': {'Actin_Reorganization': 1}, 'AnionEM_OFF': {'AnionEM': 0}, 'AnionEM_ON': {'AnionEM': 1}, 'AquaporinPIP2_1_OFF': {'AquaporinPIP2_1': 0}, 'AquaporinPIP2_1_ON': {'AquaporinPIP2_1': 1}, 'AtRAC1_OFF': {'AtRAC1': 0}, 'AtRAC1_ON': {'AtRAC1': 1}, 'CIS_OFF': {'CIS': 0}, 'CIS_ON': {'CIS': 1}, 'CPK23_OFF': {'CPK23': 0}, 'CPK23_ON': {'CPK23': 1}, 'CPK3_21_OFF': {'CPK3_21': 0}, 'CPK3_21_ON': {'CPK3_21': 1}, 'CPK6_OFF': {'CPK6': 0}, 'CPK6_ON': {'CPK6': 1}, 'Ca2_ATPase_OFF': {'Ca2_ATPase': 0}, 'Ca2_ATPase_ON': {'Ca2_ATPase': 1}, 'Ca2c_OFF': {'Ca2c': 0}, 'Ca2c_ON': {'Ca2c': 1}, 'CaIM_OFF': {'CaIM':

In [4]:
# skip_list = []

# # ignoring experiments that didn't work for ABA_GA1_A : 86
# skip_list = [
#     19,71,72,104,116,143,145,162,163,172,
#     176,177,195,197,208,210,217,221,232,236,
#     275,306,367,379,423,427,449,456,459,465,
#     480,262,270,10,35,39,66,67,82,84,
#     119,126,183,202,204,207,220,231,241,244,
#     252,254,256,267,268,282,283,288,302,307,
#     308,309,310,311,318,348,350,366,381,403,
#     406,417,425,428,430,460,461,468,471,472,
#     473,499,264,361,364,434
# ]

# ignoring experiments that didn't work for ABA_base_A : 251
skip_list = [
    18,19,63,67,79,80,98,104,116,119,
    126,128,133,143,148,153,156,157,160,172,
    176,183,195,197,200,204,207,217,221,243,
    244,260,262,294,313,326,328,331,333,355,
    367,371,373,375,377,379,381,412,414,417,
    423,425,427,428,429,434,445,454,456,465,
    480,486,35,236,267,275,345,468,473,288,
    68,73,75,77,78,88,201,208,210,212,
    213,215,232,340,449,469,474,91,92,17,
    229,257,470,177,256,270,151,352,354,356,
    368,370,374,9,10,66,84,86,117,120,
    154,165,174,175,220,241,242,252,254,306,
    307,309,310,311,318,348,350,360,362,365,
    376,397,406,418,431,437,467,499,358,261,
    263,269,264,219,308,50,93,97,140,186,
    188,190,11,89,1,13,16,21,26,28,
    30,32,39,43,45,48,56,62,74,81,
    87,95,99,103,105,107,109,115,127,132,
    134,147,150,152,155,180,196,198,199,203,
    206,211,225,230,234,245,247,249,251,253,
    255,259,280,282,284,287,293,295,299,338,
    372,382,386,398,402,409,411,416,420,422,
    424,441,453,462,471,472,475,476,478,482,
    484,488,490,492,494,496,500,502,136,138,
    161,144,164,173,226,314,347,349,439,461,
    498,
]

In [5]:
def comment_removal(line:str) -> bool:
    if line.startswith("#"):
        print(line)
    return not line.startswith("#") and not line.isspace()

def get_bo(aeon, data, data_loc, skip_list):

    dom = bonesis.aeon.AEONDomain.from_aeon_file(aeon)

    bo = bonesis.BoNesis(dom, data)


    ID, SCORE, SOURCE, PERT, NODE, VALUE = 0, 1, 2, 3, 4, 5

    file = open(data_loc, "r")
    lines = filter(comment_removal, file)
    data_csv = csv.reader(lines, delimiter="\t")

    # skip the first row
    next(data_csv)

    i = 0
    for row in data_csv:

        # ignore any data with "Some"
        if row[VALUE] in ["Some", "Some/ON", "OFF/Some"]:
            continue

        if int(row[ID]) in skip_list:
            continue
        
        i+=1

        if row[VALUE] == "ON":
            node_state = row[NODE] + "_ON"
        elif row[VALUE] == "OFF":
            node_state = row[NODE] + "_OFF"
        else:
            print("Unexpected value", row[VALUE])
            raise Exception("Unexpected experiment output")

        # wild type
        if row[SOURCE] == "" and row[PERT] == "":
            bo.fixed(bo.obs(node_state))
            bo.all_fixpoints(bo.obs(node_state))

        # perturbations
        else:
            fixes_dict = {}
            if row[SOURCE] != "":
                # add source node values to the fixes
                source_str = row[SOURCE].split(",")
                for sth in source_str:
                    node, value = sth.strip().split("=")
                    fixes_dict[node] = int(value)
        
            if row[PERT] != "":
                # add other perturbations to the fixes
                pert_str = row[PERT].split(",")
                for sth in pert_str:
                    node, value_str = sth.strip().split(" ")
                    if value_str == "KO":
                        value = 0
                    elif value_str == "CA":
                        value = 1
                    else:
                        raise Exception("Perturbation should be KO or CA")
                    fixes_dict[node] = int(value)

            with bo.mutant(fixes_dict):
                bo.fixed(bo.obs(node_state))
                bo.all_fixpoints(bo.obs(node_state))

    print("incorporated", i, "perturbations")

    return bo

In [6]:
bo = get_bo(aeon, data, data_loc, skip_list)

incorporated 167 perturbations


In [7]:
# bo.is_satisfiable()

In [8]:
model_name = "ABA_A"

i = 0
for bn in bo.boolean_networks(limit=10000):
    i += 1

    if not os.path.exists(f"{model_name}"):
        os.makedirs(f"{model_name}")
    bn.save(f"{model_name}/bn-solution_{i}.bnet")


Grounding...